In [ ]:
using BenchmarkTools
using Revise

In [ ]:
using CSV
using DataFrames
using JSON

using DataIO
using GCTGMT
using Normalization
using Support

using FeatureSetEnrichment

In [ ]:
name_to_set_to_element_ = Dict(
    splitpath(path)[end] => read_gmt(path) for
    path in read_directory("/Users/kwat/Downloads/gene_sets/")
)

;

In [ ]:
method_ = Vector{String}()

for method in keys(score_set_new(["a", "b"], [-1.0, 1.0], ["a"]; plot = false))

    push!(method_, string(method, " extreme"), string(method, " area"))

end

n_method = length(method_)

;

In [ ]:
benchmarks_directory_path = "/Users/kwat/Downloads/benchmarks/"

In [ ]:
benchmark_ = Vector{String}()

benchmark_size_ = Vector{Int64}()

compute = false

benchmark_x_method_rows = Vector{Vector{Float64}}()

;

In [ ]:
for benchmark_directory_path in read_directory(benchmarks_directory_path)

    name = splitpath(benchmark_directory_path)[end]

    println(name)

    element_, score_ =
        eachcol(read_data(joinpath(benchmark_directory_path, "gene_x_score.tsv")))

    json_dict = JSON.parse(open(joinpath(benchmark_directory_path, "gene_sets.json")))

    set_to_element_ = Dict{String,Vector{String}}()

    for gmt_path in json_dict["gene_sets_tested"]

        merge!(set_to_element_, name_to_set_to_element_[splitpath(gmt_path)[end]])

    end

    set_ = sort(collect(keys(set_to_element_)))

    n_set = length(set_)

    println("    ", n_set, " set.")

    if compute

        set_to_method_to_result = sort(score_set_new(element_, score_, set_to_element_))

        set_x_method = Matrix{Float64}(undef, n_set, n_method)

        for (set_i, (set, method_to_result)) in enumerate(set_to_method_to_result)

            set_x_method_row = []

            for result in values(method_to_result)

                append!(set_x_method_row, result[2:3])

            end

            set_x_method[set_i, :] = set_x_method_row

        end

    end

    for set in json_dict["gene_sets_positive"]

        print("    ")

        if !in(set, set_)

            println(set, " (missing)")

            continue

        end

        benchmark = string(name, '.', set)

        benchmark_size = length(set_to_element_[set])

        println(set, " (", benchmark_size, ")")

        push!(benchmark_, benchmark)

        push!(benchmark_size_, benchmark_size)

        if compute

            benchmark_x_method_row = Vector{Float64}()

            for (method_i, set_score_) in enumerate(eachcol(set_x_method))

                is_negative_ = set_score_ .< 0

                is_positive_ = 0 .<= set_score_

                set_negative_ = set_[is_negative_]

                set_positive_ = set_[is_positive_]

                negative_ =
                    .-normalize(.-set_score_[is_negative_], "1224") / sum(is_negative_)

                positive_ = normalize(set_score_[is_positive_], "1224") / sum(is_positive_)

                i_negative = findfirst(set_negative_ .== set)

                i_positive = findfirst(set_positive_ .== set)

                if i_negative != nothing

                    benchmark_score = negative_[i_negative]

                elseif i_positive != nothing

                    benchmark_score = positive_[i_positive]

                else

                    benchmark_score = set_score_[findfirst(set_ .== set)]

                    println("        ", method_[method_i], " returned ", benchmark_score)

                    #TODO: check all NaN row later

                end

                push!(benchmark_x_method_row, benchmark_score)

            end

            push!(benchmark_x_method_rows, benchmark_x_method_row)

        end

    end

end

In [ ]:
benchmark_x_method_path = "benchmark_x_method.tsv"

if compute

    benchmark_x_method = convert_vector_of_vector_to_matrix(benchmark_x_method_rows)

    benchmark_x_method = DataFrame(benchmark_x_method, method_)

    insertcols!(benchmark_x_method, 1, "Benchmark" => benchmark_)

    CSV.write(benchmark_x_method_path, benchmark_x_method)

end

benchmark_x_method = read_data(benchmark_x_method_path)

In [ ]:
benchmark_x_method_metric = Matrix{Int64}(undef, length(benchmark_), n_method)

for (benchmark_i, benchmark_score_) in
    enumerate(eachrow(select(benchmark_x_method, Not("Benchmark"))))

    classic_score = benchmark_score_["Is KS < extreme"]

    benchmark_score_ = Vector(benchmark_score_)

    if classic_score < 0

        metric_ = benchmark_score_ .< classic_score

    else

        metric_ = classic_score .< benchmark_score_

    end

    benchmark_x_method_metric[benchmark_i, :] = metric_

end

In [ ]:
function sum_sort(m::Matrix)::Nothing

    for (method, n_better) in zip(sort_like(sum(eachrow(m)), method_)...)

        println(method, "    ", n_better)

    end

    return nothing

end

In [ ]:
buffer = "="^8

benchmark_size_maximum = maximum(benchmark_size_)

println(buffer, " < ", benchmark_size_maximum, " ", buffer)

sum_sort(benchmark_x_method_metric)

increment = 50

for (minimum_size, maximum_size) in [
    ((i - 1) * increment, i * increment) for
    i = 1:Int64(ceil(benchmark_size_maximum / increment))
]

    println(buffer, " ", minimum_size, " < ", maximum_size, " ", buffer)

    is_selected = minimum_size .< benchmark_size_ .< maximum_size

    n_selected = sum(is_selected)

    println(n_selected)

    if 1 < n_selected

        sum_sort(benchmark_x_method_metric[is_selected, :])

    end

end